In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

In [3]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Conv2DTranspose
from keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [4]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

In [5]:
input_path = "F:/Kaggle/Statoil/inputs/"
batch_size = 32

In [30]:
train = pd.read_json(input_path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(input_path+"test.json")

In [ ]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [ ]:
X_train = get_scaled_imgs(train)
X_test = get_scaled_imgs(test)

In [ ]:
print("shape of train: {} and test: {}" .format(X_train.shape,X_test.shape))

In [ ]:
from clr_callback import *
from keras.optimizers import *
clr_triangular = CyclicLR(mode='triangular')

In [ ]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]
#            yield X1i[0], X1i[1]

def gen_flow_for_test(X1, X2,y=None):
    genX12 = gen.flow(X1,y, batch_size=batch_size,seed=55)
    genX22 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X21i = genX12.next()
            X22i = genX22.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X21i[0], X22i[1]]
#            yield X21i[0]
            
# Finally create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=25, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.0001)
    
    return [es,msave,reduce_lr]

In [ ]:
def getModel(p=0.1):
    
    input_shape=(75, 75, 3)
    inp = Input(shape=input_shape)
    
    bn = BatchNormalization()(inp)
    
    x = Conv2D(64, kernel_size=(3, 3),activation='relu',name="conv1")(bn)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool1")(x)
    x= Dropout(p)(x)
    
    x = Conv2D(128, kernel_size=(3, 3),activation='relu',name="conv2")(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool2")(x)
    x = Dropout(p)(x)

    x = Conv2D(256, kernel_size=(1, 1),activation='relu',name="conv3")(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool3")(x)
    x = Dropout(p)(x)

    x = Conv2DTranspose(512, kernel_size=(3, 3),activation='relu',name="conv4")(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool4")(x)
    x = Dropout(p)(x)

    x = Conv2DTranspose(256, kernel_size=(3, 3),activation='relu',name="conv5")(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool5")(x)
    x = Dropout(p)(x)

    x = Conv2DTranspose(128, kernel_size=(3, 3),activation='relu',name="conv6")(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool6")(x)
    x = Dropout(p)(x)

#     x = Conv2DTranspose(1, kernel_size=(3, 3),activation='relu',name="conv7")(x)
#     x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),name="pool7")(x)
#     x = Dropout(p)(x)
   
#    x = Conv2D(64, kernel_size=(1, 1),activation='relu',name="conv8")(x)
    x = GlobalAveragePooling2D(name='globalpool')(x)
    
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(2*p)(x)
    x = Dense(1024, activation='relu', name='fc3')(x)
    x = Dropout(2*p)(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(input=[inp], output=output)
    
    return model

In [ ]:
conv_model = getModel(p=0.1)
conv_model.summary()

In [ ]:
def angle_model():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = conv_model
    x = base_model.get_layer('globalpool').output

    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    output = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    return model


In [ ]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= angle_model()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds=myAngleCV(X_train, X_angle, X_test)

In [27]:
test.head(30)


,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",p5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",p4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",pb20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",pe7f018bb,34.473900
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",p4371c8c3,43.918874
5,"[-18.13534927368164, -18.13534927368164, -18.1...","[-14.650325775146484, -14.650325775146484, -14...",pa8d9b1fd,35.933020
6,"[-24.565174, -26.567524, -25.946882, -26.89542...","[-32.915886, -35.194798, -32.272282, -28.7505,...",p29e7727e,42.414200
7,"[-20.162964, -18.16898, -17.023144, -17.023216...","[-30.905346, -28.406647, -25.381365, -28.77645...",p92a51ffb,33.638300
8,"[-21.920645, -21.737827, -21.383303, -21.73792...","[-25.53537, -28.514198, -27.75848, -31.280354,...",pc769ac97,41.107600
9,"[-14.811565, -16.318594, -17.082613, -16.41037...","[-22.617977, -21.065811, -27.914198, -25.24334...",paee0547d,34.966400


In [31]:
test[["id"]] = "'"+test[['id']].astype(str) 

In [32]:
pd.DataFrame(test['id'].values).to_csv('F:/Kaggle/Statoil/submit/id.csv')

In [ ]:
#Submission
import datetime as dt

submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('F:/Kaggle/Statoil/submit/sub5cv_cnn_withangle_deep'+dt.datetime.today().strftime("%d%m%Y%H%M")+'.csv', index=False)